# Theramex
* Travel and Accommodation separatly
* Subtract self paid?

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [4]:
columns = [49.4, 157.8, 171.5, 196.7, 218.8, 262, 344, 374.6, 486.1, 518.7, 536.6, 564, 590.3,  698.6]
df = tabula.read_pdf("2019-transparency-report-for-Swiss-HCPs.pdf", pages='all', lattice=False, columns=columns, guess=False, pandas_options={'header': None})

## Format Table

In [37]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'type', 'empty1', 'country', 'location', 'address', 'uci', 'empty2', 'registration_fees', 'travel', 'accommodation', 'fees', 'selfpaid', 'total']

#Remove empty rows
df_export = df_export.dropna(subset=['name', 'country'], how='all')
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.dropna(subset=['name', 'country'], how='all')

#Lowercase type
df_export['type'] = df_export['type'].str.lower()

#Copy PLZ from Location to PLZ
df_export['plz'] = df_export.location.str.extract(r', (\d{4})')
df_export['location'] = df_export.location.str.replace(r'(, \d{4})', '')

#Add missing rows
df_export['donations_grants'] = np.nan
df_export['sponsorship'] = np.nan
df_export['travel_accommodation'] = np.nan
df_export['related_expenses'] = np.nan

#Clean numbers
df_export = cleanup_number(df_export)
df_export = remove_chf(df_export)
df_export = remove_comma(df_export)

#Clean Travel and Accommodation separatly
df_export['travel'] = df_export['travel'].str.replace('CHF ', '')
df_export['travel'] = df_export['travel'].str.replace(',', '')
df_export['accommodation'] = df_export['accommodation'].str.replace('CHF ', '')
df_export['accommodation'] = df_export['accommodation'].str.replace(',', '')
#df_export['selfpaid'] = df_export['selfpaid'].str.replace('CHF ', '')
#df_export['selfpaid'] = df_export['selfpaid'].str.replace(', ', '')
#df_export['selfpaid'] = df_export['selfpaid'].str.replace('Not required', '')

#Calc travel_accommodation
df_export['travel_accommodation'] = df_export.apply(lambda row: float(row['accommodation']) + float(row['travel']), axis=1)

#Drop
df_export.drop(columns=['empty0', 'empty1', 'empty2', 'travel', 'accommodation', 'selfpaid'], inplace=True)

#Remove Titles
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Amounts to number
df_export = amounts_to_number(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'theramex')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', True)